In [1]:
pip install requests pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gspread oauth2client


Note: you may need to restart the kernel to use updated packages.


In [7]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Use the same scope as before
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# Use your downloaded json file with your Google API credentials
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\gkers\Documents\INTECH\INTECH_GUI\INTECH_GUI\intechgui-68cb3ebdb2d2.json", scope)
client = gspread.authorize(creds)

# Access the Google Sheet using the URL
sheet_url = "https://docs.google.com/spreadsheets/d/10hzD9efmuT1bAeipZc3cwfAl8s7nlxu2ZbFbX1bWMQU/edit#gid=0"  # Make sure to share your Google sheet with the client_email from the JSON file
sheet = client.open_by_url(sheet_url)

# Choose the right sheet or worksheet if there are multiple ones
worksheet = sheet.get_worksheet(0)  # `0` refers to the first sheet within the spreadsheet

# Get all the values from the sheet into a DataFrame
data = worksheet.get_all_values()
headers = data.pop(0)  # Assumes first row is header
df = pd.DataFrame(data, columns=headers)

# Convert 'Date and time (UTC)' to datetime and sort it
df['DateTime'] = pd.to_datetime(df['Time'], format='%y%m%d%H%M%S', errors='coerce')
print(df)


  Node Identifer          Time Water Height Battery Voltage Panel Power  \
0           N001  240409234459         None               0           0   
1           N001  240412184459         None               0           0   
2           N001  240412185959  1.585594577               0           0   
3           N001  240412191459  4.240384915               0           0   
4           N001  240412184459         None               0           0   
5           N001  240412185959  1.585594577               0           0   
6           N001  240412191459  4.240384915               0           0   
7           N002  240415205959         None           13.68           4   

  Processor Temp Sleep Mode            DateTime  
0          37.01          1 2024-04-09 23:44:59  
1          47.24          1 2024-04-12 18:44:59  
2          42.93          1 2024-04-12 18:59:59  
3          45.08          1 2024-04-12 19:14:59  
4          47.24          1 2024-04-12 18:44:59  
5          42.93        

In [4]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

# Update figure properties for better visibility and formatting
def update_figure_layout(figure):
    figure.update_traces(line=dict(color='white', width=2))  # Set line color to white and increase thickness
    figure.update_xaxes(tickangle=-45, tickformat='%d %b\n%H:%M')  # Format x-axis labels
    figure.update_yaxes(tickfont=dict(color='white'))  # Set y-axis tick font color to white for better visibility
    figure.update_layout(
        paper_bgcolor="#333",
        plot_bgcolor="#333",
        font=dict(color="#FFF"),
        xaxis_showgrid=False,
        yaxis_showgrid=True,
        yaxis_gridcolor="#555",
        margin=dict(l=40, r=40, t=40, b=40)
    )
    return figure

# Hardcoded latitude and longitude for the sensor
sensor_latitude = 40.085177  # Replace with actual latitude
sensor_longitude = -105.219921  # Replace with actual longitude

app = dash.Dash(__name__)

app.layout = html.Div(style={
    'backgroundColor': '#333',
    'color': '#DDD',
    'padding': '10px',
    'display': 'grid',
    'gridTemplateColumns': '1fr 1fr',
    'gridTemplateRows': '1fr 1fr',
    'gap': '10px',
    'height': '90vh'
    }, children=[
    
    # Map in the top left with hardcoded latitude and longitude
    dcc.Graph(
        id='map',
        figure=px.scatter_mapbox(
            lat=[sensor_latitude],
            lon=[sensor_longitude],
            zoom=11,
            height=300,
            text = ['Node 1'],
            hover_name=['Node 1'],  # Names appear in bold in the hover tooltip
        ).update_layout(
            mapbox_style="dark",
            mapbox_accesstoken="pk.eyJ1IjoiZ2tlcnNleTEwNiIsImEiOiJjbHV1OGY2cXcwN25wMmxvOTAxbXB3azYxIn0.ppr7W_ticC4BF2iHyN7VYw",  # Replace with your actual Mapbox access token
            margin={"r":0,"t":0,"l":0,"b":0},
            paper_bgcolor="#444",
            plot_bgcolor="#444",
            font=dict(color="#DDD")
        ),
        style={'gridArea': '1 / 1 / 2 / 2'}
    ),
    
    # Water Level Data in the top right
    dcc.Graph(
        id='water-level',
        figure=px.line(
            df,
            x='Date and Time (Mountain Time)',
            y='Water Level',
            title='Water Level Over Time'
        ).update_layout(
            paper_bgcolor="#444",
            plot_bgcolor="#444",
            font=dict(color="#DDD")
        ),
        style={'gridArea': '1 / 2 / 2 / 3'}
    ),
    
    # Battery Voltage in the bottom left
    dcc.Graph(
        id='battery-voltage',
        figure=px.line(
            df,
            x='Date and Time (Mountain Time)',
            y='Battery Voltage',
            title='Battery Voltage Over Time'
        ).update_layout(
            paper_bgcolor="#444",
            plot_bgcolor="#444",
            font=dict(color="#DDD")
        ),
        style={'gridArea': '2 / 1 / 3 / 2'}
    ),
    
    # PV Input in the bottom right
    dcc.Graph(
        id='pv-input',
        figure=px.line(
            df,
            x='Date and Time (Mountain Time)',
            y='PV Input',
            title='PV Input Over Time'
        ).update_layout(
            paper_bgcolor="#444",
            plot_bgcolor="#444",
            font=dict(color="#DDD")
        ),
        style={'gridArea': '2 / 2 / 3 / 3'}
    ),    
])

if __name__ == '__main__':
    app.run_server(debug=True)


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Node Identifer', 'Time', 'Water Height', 'Battery Voltage', 'Panel Power', 'Processor Temp', 'Sleep Mode'] but received: Date and Time (Mountain Time)